In [1]:
# MN is using Kernel Python 3.10.13 for this module

# In this weeks Python sessions we’ll look at how we can feed our data into some ML code to get
# a decision tree going which can help us with categorisation problems, but in a way that’s maybe,
# more visually tractable.

# Here’s a link, again, to the google developers course on machine learning, but specifically to
# decision trees: https://developers.google.com/machine-learning/decision-forests/decision-trees
# It’s not required reading but it’ll help and will clear up things that I’m not always the best
# at explaining live, whilst also trying to remember code!

In [2]:
import pandas as pd
import numpy as np

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.impute import SimpleImputer

In [ ]:
col_names = ['pregancies', 'glu', 'bp', 'skin', 'insulin', 'bmi', 'pedigree', 'age', 'label']

df = pd.read_csv('https://raw.githubusercontent.com/data-to-insight/ERN-sessions/main/data/diabetes_data.csv',
                 names=col_names,
                 skiprows=1)
 
df.head()

In [ ]:
imp = SimpleImputer(strategy='mean')
 
error_cols = col_names[1:-1]
 
df[error_cols] =df[error_cols].replace(0, np.nan)
 
for column in error_cols:
    df[column] = imp.fit_transform(df[column].values.reshape(-1, 1))
 
df.head(10)

In [ ]:
features = col_names[:-1]
X = df[features]
y = df['label']

X.head(10)

# https://scikit-learn.org/stable/modules/impute.html

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.3,
                                                    random_state=1)
 
clf = DecisionTreeClassifier()
 
clf.fit(X_train, y_train)
 
y_pred = clf.predict(X_test)

In [7]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
cm = metrics.confusion_matrix(y_test, y_pred)
 
target_values = ['no diabetes', 'diabetes']
 
sns.heatmap(cm,
            annot=True,
            xticklabels=target_values,
            yticklabels=target_values)
plt.xlabel('Predicted values')
plt.ylabel('Real values')
plt.title('Confusion Matrix')

In [ ]:
print(f'Diabetes precision {51/82*100}')
print(f'Diabetes recall {51/85*100}')
print()
print(f'No Diabetes precision {120/154*100}')
print(f'No Diabetes recall {120/151*100}')

In [ ]:
from sklearn.metrics import classification_report
output_report = classification_report(y_test, y_pred, target_names=target_values,
                                      output_dict=True)
 
output_report

In [ ]:
# IN THE TERMINAL, run these 3 lines 1-by-1
# pip install pydotplus
# pip install graphviz
# conda install graphviz (not sure about this one)

# alternatively, in the codespace write the following:
# %pip install graphviz
# %pip install pydotplus
# %conda install graphviz

# "we have to install graphviz twice, this is how I could get it to work, I don't know why this is the case" Will
# if I do the "%conda install" it gives me a new error, so I'm not doing that.


from sklearn.tree import export_graphviz
from six import StringIO
from IPython.display import Image
import pydotplus
 
y_pred = clf.predict(X_test)

dot_data = StringIO()
export_graphviz(clf,
                out_file=dot_data,
                filled=True,
                rounded=True,
                special_characters=True,
                feature_names=features,
                class_names=target_values)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())

graph.write_png('diabetes.png')
Image(graph.create_png())

In [67]:
# this last bit of code has a problem, "GraphViz's executables not found"

# https://github.com/data-to-insight/D2I-Jupyter-Notebook-Tools/blob/main/ml-data%20science%20tutorials/decision%20tree.ipynb

# see module2_4b.ipynb where the exercise from Will (at link above) has been copied & tested by MN.
# in that test there is still a problem with finding the graphviz executables

# Declan is using the same kernel (Python 3.10.13) but he is not having the same Graphviz problems.
# The only difference we can see is that in Declan's terminal the prompt does not say:
# (.conda) @MarkNunns ➜ /workspaces/Python_ESCC_course (main) $ 
# Declan's prompt just shows his username, that's it


# Verify the environment
# https://coderefinery.github.io/installation/conda-environment/

# Declan is using the same kernel (Python 3.10.13) in the top right corner.
# Also, in the terminal when DF checks the Python version (using: "python --version") it says Python 3.10.13.

# For MN, it says Python 3.10.13 in the top right corner, but
# in the terminal when MN checks the Python version it says 3.12.5
# if I can get this changed, the code may start to work better.

In [ ]:
# accuracy': 0.7186147186147186 ??

clf = DecisionTreeClassifier(max_depth=3, criterion='entropy')

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(metrics.accuracy_score(y_test, y_pred))

output_report

In [ ]:
# The second attribute selection measure we looked at was entropy.
# Entropy is a measure of how homogenous the samples in a node are, or how random
# the samples in the node are. It can be though of as how messy the data is in a given node,
# or how much information our node gives us about the outcomes we want to predict.

# High entropy means the system is messy and doesn’t give us a lot of information about a prediction.
# When we use entropy as our splitting criteria, which we did for the second plot, this choses
# to split nodes will maximise the decrease in average entropy between
# nodes (this difference in average entropy is called information gain).
# Essentially then, when choosing entropy as our attribute selection method,
# we split nodes where, after that split, we have a more homogenous set of samples,
# which can be though of as having a better set of samples from which to make a prediction.